In [1]:
# import the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
import shutil
from scipy import ndimage
# from joblib import Parallel, delayed
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [2]:
sub_window_and_count_path = 'final_train_sub_windows_and_counts'
# load the data

# subwindow data
X_train = np.load('final_train_sub_windows_and_counts/train_sub_windows.npy')
print(X_train.shape)

# sub counts data
y_train = np.load('final_train_sub_windows_and_counts/train_sub_counts.npy')
print(y_train.shape)

# subwindow data
X_valid = np.load('final_valid_sub_windows_and_counts/valid_sub_windows.npy')
print(X_valid.shape)

# sub counts data
y_valid = np.load('final_valid_sub_windows_and_counts/valid_sub_counts.npy')
print(y_valid.shape)

(1572864, 32, 32, 3)
(1572864,)
(393216, 32, 32, 3)
(393216,)


In [3]:
# load and add the new layers to the prev trained model
TN_model = tf.keras.models.load_model('../models/tasselnet1_overlapping_w32.h5')
# look at the input shape
TN_model.input
# need to remove some of the last layers
output_trial = TN_model.layers[-8].output
    
# create the functional API model
reduced_model = tf.keras.models.Model(TN_model.input, output_trial)

In [4]:
reduced_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_input (InputLayer)   [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                             

In [5]:
# add dropout
added_dropout = tf.keras.layers.Dropout(0.5, name = "New_dropout")(TN_model.layers[-8].output)

# add flatten
added_flatten = tf.keras.layers.Flatten(name = "Flatten2")(added_dropout)

# add dense
added_dense = tf.keras.layers.Dense(512, name = "New_Dense")(added_flatten)

# add activation
added_Act = tf.keras.layers.Activation('relu', name = "New_Activation")(added_dense)

# add dropout
added_dropout2 = tf.keras.layers.Dropout(0.5, name = "New_dropout2")(added_Act)

# add dense
added_dense2 = tf.keras.layers.Dense(1, name = "New_Dense2")(added_dropout2)

# add activation
added_Act2 = tf.keras.layers.Activation('relu', name = "New_Activation2")(added_dense2)

# define the new model with functional API
new_model = tf.keras.models.Model(TN_model.input, added_Act2)


In [6]:
def data_generator(x_data, y_data, batch_size, shuffle=False, peek=False, verbose=False):
    num_samples = len(x_data)
    indices = np.arange(num_samples)
    
    if peek:    # Give first batch unshuffled and don't change start index when peeking for training
        end = min(batch_size, num_samples)
        if verbose:
            print(f"Generating peeking batch up to index {end}")
        batch_x = x_data[:end]
        batch_y = y_data[:end]
        peek = False
        yield (batch_x, batch_y)

    while True:    # Loop indefinitely for epochs
        # Shuffle indices at the start of each epoch after the peek, if shuffle is enabled
        if shuffle:
            np.random.shuffle(indices)
        
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            batch_indices = indices[start:end]

            # Print batch indices if verbose
            if verbose:
                # Warning: calling verbose when shuffling will usually clutter output
                if shuffle:
                    if len(batch_indices) < 16:
                        print(f"Batch indices: {np.sort(batch_indices)}")
                    else:
                        print(f"Printing batch indices would clutter output. Skipped.")
                    print(f"Length: {len(batch_indices)}")
                else:
                    print(f"Generating batch from index {start} to {end}")
                # traceback.print_stack()

            # Generate batches
            batch_x = x_data[batch_indices]
            batch_y = y_data[batch_indices]

            # Yield the current batch
            yield (batch_x, batch_y)

In [7]:
%%time
batch_size = 1024
generator_batch_size = 1024
train_gen = data_generator(X_train, y_train, batch_size, shuffle=False, peek=True, verbose=False)
val_gen = data_generator(X_valid, y_valid, generator_batch_size, shuffle=False, peek=True, verbose=False)

CPU times: user 1 µs, sys: 6 µs, total: 7 µs
Wall time: 11.4 µs


In [8]:
steps_per_epoch = int(np.ceil(len(X_train) / batch_size))
train_steps = int(np.ceil(len(X_train) / generator_batch_size))
validation_steps = int(np.ceil(len(X_valid) / generator_batch_size))

In [9]:
#     freeze layers
for layer in TN_model.layers:
    layer.trainable = False
    
# compile the model
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
new_model.compile(loss='mean_squared_error', optimizer=opt, metrics = ['mean_absolute_error'])
    
# add early stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights = True, verbose=1, patience=10)

#     fit the frozen model:
history = new_model.fit(train_gen,
          validation_data = val_gen, steps_per_epoch=steps_per_epoch, 
                                  validation_steps=validation_steps,                        
          epochs = 100, callbacks = [es])

Epoch 1/100
1536/1536 [==============================] - 28s 15ms/step - loss: 0.0262 - mean_absolute_error: 0.0627 - val_loss: 0.0206 - val_mean_absolute_error: 0.0340
Epoch 2/100
1536/1536 [==============================] - 24s 15ms/step - loss: 0.0252 - mean_absolute_error: 0.0613 - val_loss: 0.0203 - val_mean_absolute_error: 0.0351
Epoch 3/100
1536/1536 [==============================] - 23s 15ms/step - loss: 0.0249 - mean_absolute_error: 0.0614 - val_loss: 0.0200 - val_mean_absolute_error: 0.0375
Epoch 4/100
1536/1536 [==============================] - 23s 15ms/step - loss: 0.0247 - mean_absolute_error: 0.0616 - val_loss: 0.0198 - val_mean_absolute_error: 0.0391
Epoch 5/100
1536/1536 [==============================] - 24s 16ms/step - loss: 0.0246 - mean_absolute_error: 0.0614 - val_loss: 0.0197 - val_mean_absolute_error: 0.0396
Epoch 6/100
1536/1536 [==============================] - 23s 15ms/step - loss: 0.0245 - mean_absolute_error: 0.0612 - val_loss: 0.0195 - val_mean_absolute_

In [10]:
# # unfreeze few layers and retrain - finetune the model
TN_model.trainable = True

set_trainable = False

for layer in TN_model.layers:
    if layer.name == 'conv2d_3':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
            
    # # compile the mdoel
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
new_model.compile(loss='mean_squared_error', optimizer=opt, metrics = ['mean_absolute_error'])
    
    # # add early stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights = True, verbose=1, patience=10)
    
    # # fit the model (fine tuned)
history_new = new_model.fit(train_gen,
          validation_data = val_gen, steps_per_epoch=steps_per_epoch, 
                                  validation_steps=validation_steps,                        
          epochs = 100, callbacks = [es])


Epoch 1/100
1536/1536 [==============================] - 26s 16ms/step - loss: 0.0223 - mean_absolute_error: 0.0531 - val_loss: 0.0186 - val_mean_absolute_error: 0.0376
Epoch 2/100
1536/1536 [==============================] - 25s 16ms/step - loss: 0.0218 - mean_absolute_error: 0.0521 - val_loss: 0.0186 - val_mean_absolute_error: 0.0386
Epoch 3/100
1536/1536 [==============================] - 26s 17ms/step - loss: 0.0215 - mean_absolute_error: 0.0516 - val_loss: 0.0182 - val_mean_absolute_error: 0.0379
Epoch 4/100
1536/1536 [==============================] - 26s 17ms/step - loss: 0.0212 - mean_absolute_error: 0.0512 - val_loss: 0.0181 - val_mean_absolute_error: 0.0368
Epoch 5/100
1536/1536 [==============================] - 25s 16ms/step - loss: 0.0211 - mean_absolute_error: 0.0511 - val_loss: 0.0177 - val_mean_absolute_error: 0.0367
Epoch 6/100
1536/1536 [==============================] - 26s 17ms/step - loss: 0.0209 - mean_absolute_error: 0.0508 - val_loss: 0.0178 - val_mean_absolute_

In [11]:
# save the finetuned model
model_name = 'TN_model_with_new_data.h5'
new_model.save('models' + '/' + model_name)